<a href="https://colab.research.google.com/github/HYUN816/python_data_eng/blob/main/data08%20-%20%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C/%EC%98%81%ED%99%94%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise

In [ ]:
import surprise

In [ ]:
print(surprise.__version__)

1.1.1


In [ ]:
from surprise import SVD ##고유값분해(행렬을 더 낮은 차원으로 분해)해서 다시 원본으로 살리면서 잠재적인 값을 추론하는 방법
from surprise import Dataset, Reader ## SVD를 사용하기 위한 데이터셋 만들어주는 클래스 / ## 사용자, 아이템, 평점
from surprise import accuracy ## RMSE, MAE, R2, CV(k-fold)
from surprise.model_selection import train_test_split ### 검증/테스트 데이터 분류

In [ ]:
## 1. 데이터셋을 만들어 주자(사용자, 아이템, 평점), 검증/테스트 데이터 분류 
data = Dataset.load_builtin('ml-100k')

In [ ]:
trainset, testset = train_test_split(data, test_size = 0.25, random_state=0)

In [ ]:
trainset

In [ ]:
testset[:5]

[('120', '282', 4.0),
 ('882', '291', 4.0),
 ('535', '507', 5.0),
 ('697', '244', 5.0),
 ('751', '385', 4.0)]

In [ ]:
## 2. SVD() 모델 선정
algo = SVD()

In [ ]:
## 3. 훈련용 데이터로 fit()
algo.fit(trainset)

In [ ]:
## 4. 검증용 데이터로 예측
predictions = algo.test(testset)[:5]

In [ ]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions]

[('120', '282', 3.6213352950262325),
 ('882', '291', 3.6700680163645067),
 ('535', '507', 4.074847873736931),
 ('697', '244', 3.3834940980814183),
 ('751', '385', 3.1949106551515336)]

In [ ]:
## 5. 정확도 계산

In [ ]:
## 6. 사용자, 아이템 정보 주고 예측 실행 
uid = str(196)
iid = str(302)
pred = algo.predict(uid,iid)
pred

Prediction(uid='196', iid='302', r_ui=None, est=4.427578829609807, details={'was_impossible': False})

In [ ]:
import pandas as pd

In [ ]:
## csv를 SVD 하기 위한 DataSet을 주기 위해서는 
## 1) df <----- csv, 2) Dataset <------ df

In [ ]:
ratings = pd.read_csv('/content/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
reader = Reader(rating_scale =(0.5, 5.0))

In [ ]:
## svd에서 사용할 수 있는 데이터 셋으로 만들어주자
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

In [ ]:
## 데이터 분류, 객체생성, 훈련, 검증 결과 print

In [ ]:
trainset, testset = train_test_split(data,test_size = 0.25, random_state=0)

In [ ]:
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8710


0.8710183111503011

In [ ]:
from surprise.model_selection import cross_validate

In [ ]:
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose =True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8789  0.8769  0.8653  0.8756  0.8784  0.8750  0.0050  
MAE (testset)     0.6747  0.6735  0.6640  0.6739  0.6743  0.6721  0.0041  
Fit time          9.45    9.28    10.59   5.04    5.06    7.88    2.36    
Test time         0.58    0.29    0.45    0.39    0.15    0.37    0.15    


{'fit_time': (9.450012922286987,
  9.276583671569824,
  10.591615915298462,
  5.035592555999756,
  5.062147855758667),
 'test_mae': array([0.67473868, 0.67352339, 0.66398404, 0.67385969, 0.67426124]),
 'test_rmse': array([0.87885504, 0.87693401, 0.86526529, 0.87558787, 0.87836891]),
 'test_time': (0.5839014053344727,
  0.28869199752807617,
  0.44672369956970215,
  0.38997316360473633,
  0.15337538719177246)}

In [ ]:
movies = pd.read_csv('/content/movies (1).csv')

In [ ]:
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds == 42].count() == 0 :
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [ ]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.50   {'was_impossible': False}


In [ ]:
## 사용자가 안본 영화 리스트를 구해서 이중에서 추천하기

def get_unseen_surprise(movies, ratings, userId) :
  # 전체영화 리스트 
  total_movies =movies['movieId'].tolist()

  # 내가 본 영화 리스트 
  seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()

  # 추천 대상이 되는 영화 리스트 = 전체영화리스트 - 내가 본 영화리스트
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]

  print('전체영화리스트 개수 >> ', len(total_movies))
  print('내가 본 영화 리스트 개수 >>', len(seen_movies))
  print('내가 안본 영화 리스트 개수>>', len(unseen_movies))

  return unseen_movies

In [ ]:
unseen_movies = get_unseen_surprise(movies, ratings, 9)

전체영화리스트 개수 >>  9742
내가 본 영화 리스트 개수 >> 46
내가 안본 영화 리스트 개수>> 9696


In [ ]:
# 클래스안에 클래스 껴놓는것 -> 멤버변수를 공유하고 싶어서 다만 독립적으로 클래스를 사용할 수 없음

In [ ]:
def recommend_movie_surprise(algo, userId, unseen_movies, top_n=5):
  # 안본 영화 리스트를 하나씩 꺼낸 다음 평점 예측
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  def sortkey_est(one):
    return one.est

  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]
  
  top_movies_ids = [int(pred.iid) for pred in top_predictions]
  top_movies_ratings = [pred.est for pred in top_predictions]
  top_movies_title = movies[movies.movieId.isin(top_movies_ids)]['title']
  top_movies_preds = [(id, title,rating) for id, title, rating in zip(top_movies_ids, top_movies_title,top_movies_ratings)]

  return top_movies_preds

In [ ]:
top_predictions = recommend_movie_surprise(algo,9, unseen_movies)
top_predictions

[(1, 'Toy Story (1995)', 3.498940113302508),
 (2, 'Jumanji (1995)', 3.498940113302508),
 (3, 'Grumpier Old Men (1995)', 3.498940113302508),
 (4, 'Waiting to Exhale (1995)', 3.498940113302508),
 (5, 'Father of the Bride Part II (1995)', 3.498940113302508)]

### 새로운 파일로 추천 시스템 만들기

In [ ]:
rating_df =pd.read_csv('/content/movie_rating.csv')
rating_df.columns=['userId','movieId','rating']
rating_df

,userId,movieId,rating
0,Jack,Lady,3.0
1,Jack,Snakes,4.0
2,Jack,You Me,3.5
3,Jack,Superman,5.0
4,Jack,The Night,3.0
5,Mick,Lady,3.0
6,Mick,Snakes,4.0
7,Mick,Just My,2.0
8,Mick,Superman,3.0
9,Mick,You Me,2.0


In [ ]:
import numpy as np
movie_li = list(set(rating_df.title.tolist()))
# id = [i+1 for i in range(len(movie_li))]

In [ ]:
movie_df =pd.DataFrame({'movieId':movie_li})

In [ ]:
movie_df

,movieId
0,The Night
1,Lady
2,Just My
3,Snakes
4,Superman
5,You Me


In [ ]:
reader = Reader(rating_scale =(0.5, 5.0))

In [ ]:
data = Dataset.load_from_df(rating_df, reader)

In [ ]:
trainset1, testset1 = train_test_split(data,test_size = 0.25, random_state=0)

In [ ]:
algo = SVD()
algo.fit(trainset1)
pred1 = algo.test(testset1)
accuracy.rmse(pred1)

RMSE: 0.8748


0.8747646762022275

In [ ]:
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose =True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2397  0.7993  0.5399  0.6412  1.1643  0.8769  0.2790  
MAE (testset)     0.9969  0.5173  0.4440  0.5832  0.9294  0.6942  0.2250  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'fit_time': (0.0017879009246826172,
  0.002578258514404297,
  0.002259969711303711,
  0.0015711784362792969,
  0.0014445781707763672),
 'test_mae': array([0.99687628, 0.51731173, 0.44401205, 0.58323452, 0.92944148]),
 'test_rmse': array([1.23973167, 0.79931695, 0.53994465, 0.64122768, 1.16427221]),
 'test_time': (0.0015778541564941406,
  0.0011508464813232422,
  0.00010013580322265625,
  7.081031799316406e-05,
  5.984306335449219e-05)}

In [ ]:
unseen_movie_toby = get_unseen_surprise(movie_df, rating_df,'Toby')

전체영화리스트 개수 >>  6
내가 본 영화 리스트 개수 >> 3
내가 안본 영화 리스트 개수>> 3


In [ ]:
def recommend_movie_surprise(algo, userId, top_n=5):
  # 안본 영화 리스트를 하나씩 꺼낸 다음 평점 예측
  unseen_movies=get_unseen_surprise(movie_df, rating_df,userId)
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  def sortkey_est(one):
    return one.est

  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]
  
  top_movies_ids = [pred.iid for pred in top_predictions]
  top_movies_ratings = [pred.est for pred in top_predictions]
  top_movies_preds = [(id,rating) for id, rating in zip(top_movies_ids, top_movies_ratings)]

  return top_movies_preds

In [ ]:
recommend_movie_surprise(algo, 'Jack')

전체영화리스트 개수 >>  6
내가 본 영화 리스트 개수 >> 5
내가 안본 영화 리스트 개수>> 1


[('Just My', 3.1045680024189086)]